# Synodal Bible & Mother

## Summary

In [1]:
# Code summary of below

In [2]:
!python --version

Python 3.10.13


## Import

### Libraries

In [3]:
from backend import *
import multiprocessing as mp
from multiprocessing import Pool, Process
import pickle
num_cores = int(mp.cpu_count()-2)
print(f"Number of cores : {num_cores}")

Number of cores : 6


In [4]:
%matplotlib inline

### Definitions

In [5]:
texts = '../texts/fiction/utf8/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', \
             'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

In [6]:
motherTokenDf = pd.read_pickle('./proc/MotherTokendf.pkl')
bibleTokenDf = pd.read_pickle('./proc/BibleTokenDf.pkl')
confessionTokenDf = pd.read_pickle('./proc/ConfessionTokenDf.pkl')

In [7]:
good_pos = ['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN', 'NUM']

In [8]:
good_pos = ['NOUN', 'ADJ', 'VERB', 'PROPN']

In [9]:
# add stopword boolean, True if all attribute columns are null
#motherTokenDf['stopword'] = ~motherTokenDf.loc[:, tokenCols[9:]].any(axis=1)
motherTokenDf['stopword'] = ~motherTokenDf.pos.isin(good_pos)
TokenDfIdx = pd.Index(range(1, (motherTokenDf.shape[0]+1)), name='id')
motherTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

bibleTokenDf['stopword'] = ~bibleTokenDf.pos.isin(good_pos)
TokenDfIdx = pd.Index(range(1, (bibleTokenDf.shape[0]+1)), name='id')
bibleTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

confessionTokenDf['stopword'] = ~confessionTokenDf.pos.isin(good_pos)
TokenDfIdx = pd.Index(range(1, (confessionTokenDf.shape[0]+1)), name='id')
confessionTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

## Bible and Mother Token Dataframes

In [10]:
bibleTokenDf

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,В,1_1,1_2,case,ADP,в,None,...,None,None,None,None,None,None,None,None,None,True
2,1,2,8,начале,1_2,1_3,obl,NOUN,начало,Inan,...,Loc,None,Neut,None,Sing,None,None,None,None,False
3,1,9,17,сотворил,1_3,1_0,root,VERB,сотворить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False
4,1,18,21,Бог,1_4,1_3,nsubj,PROPN,бог,Anim,...,Nom,None,Masc,None,Sing,None,None,None,None,False
5,1,22,26,небо,1_5,1_3,obj,NOUN,небо,Inan,...,Acc,None,Neut,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675064,37086,32,38,Христа,1_5,1_4,flat:name,PROPN,христос,Anim,...,Gen,None,Masc,None,Sing,None,None,None,None,False
675065,37086,39,41,со,1_6,1_8,case,ADP,с,None,...,None,None,None,None,None,None,None,None,None,True
675066,37086,42,47,всеми,1_7,1_8,det,DET,весь,None,...,Ins,None,None,None,Plur,None,None,None,None,True


In [11]:
bibleTokenDf.loc[bibleTokenDf.stopword == False]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
2,1,2,8,начале,1_2,1_3,obl,NOUN,начало,Inan,...,Loc,None,Neut,None,Sing,None,None,None,None,False
3,1,9,17,сотворил,1_3,1_0,root,VERB,сотворить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False
4,1,18,21,Бог,1_4,1_3,nsubj,PROPN,бог,Anim,...,Nom,None,Masc,None,Sing,None,None,None,None,False
5,1,22,26,небо,1_5,1_3,obj,NOUN,небо,Inan,...,Acc,None,Neut,None,Sing,None,None,None,None,False
7,1,29,34,землю,1_7,1_5,conj,NOUN,земля,Inan,...,Acc,None,Fem,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675060,37086,0,9,Благодать,1_1,1_2,nsubj,NOUN,благодать,Inan,...,Nom,None,Fem,None,Sing,None,None,None,None,False
675061,37086,10,17,Господа,1_2,1_1,appos,PROPN,господь,Anim,...,Gen,None,Masc,None,Sing,None,None,None,None,False
675063,37086,25,31,Иисуса,1_4,1_1,nmod,PROPN,иисус,Anim,...,Gen,None,Masc,None,Sing,None,None,None,None,False


In [12]:
motherTokenDf#.set_index(['p_id','token_id'])

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
1,1,7,13,Каждый,1_1,1_2,det,DET,каждый,None,...,Acc,None,Masc,None,Sing,None,None,None,None,True
2,1,14,18,день,1_2,1_28,obl,NOUN,день,Inan,...,Acc,None,Masc,None,Sing,None,None,None,None,False
3,1,19,22,над,1_3,1_5,case,ADP,над,None,...,None,None,None,None,None,None,None,None,None,True
4,1,23,30,рабочей,1_4,1_5,amod,ADJ,рабочий,None,...,Ins,Pos,Fem,None,Sing,None,None,None,None,False
5,1,31,40,слободкой,1_5,1_28,obl,NOUN,слободка,Inan,...,Ins,None,Fem,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86005,4436,7,13,Кто-то,1_1,1_2,nsubj,PRON,кто-то,None,...,Nom,None,None,None,None,None,None,None,None,True
86006,4436,14,21,ответил,1_2,1_0,root,VERB,ответить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False
86007,4436,22,24,ей,1_3,1_2,iobj,PRON,она,None,...,Dat,None,Fem,None,Sing,3,None,None,None,True


In [13]:
motherTokenDf.loc[motherTokenDf.stopword == False]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
2,1,14,18,день,1_2,1_28,obl,NOUN,день,Inan,...,Acc,None,Masc,None,Sing,None,None,None,None,False
4,1,23,30,рабочей,1_4,1_5,amod,ADJ,рабочий,None,...,Ins,Pos,Fem,None,Sing,None,None,None,None,False
5,1,31,40,слободкой,1_5,1_28,obl,NOUN,слободка,Inan,...,Ins,None,Fem,None,Sing,None,None,None,None,False
7,1,44,50,дымном,1_8,1_34,conj,NOUN,дымный,Inan,...,Loc,None,Masc,None,Sing,None,None,None,None,False
8,1,52,60,масляном,1_10,1_11,amod,VERB,масляный,None,...,Loc,None,Masc,None,Sing,None,Pres,Part,Act,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86003,4434,54,61,хрипела,2_2,2_0,root,ADJ,хрипеть,None,...,None,Pos,Fem,None,Sing,None,None,None,None,False
86004,4435,10,20,Несчастные,1_2,1_0,root,ADJ,несчастный,None,...,Nom,Pos,None,None,Plur,None,None,None,None,False
86006,4436,14,21,ответил,1_2,1_0,root,VERB,ответить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False


In [14]:
confessionTokenDf

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
1,1,10,19,Позвольте,1_2,1_0,root,VERB,позволить,None,...,None,None,None,Imp,Plur,2,None,Fin,Act,False
2,1,20,30,рассказать,1_3,1_2,xcomp,VERB,рассказать,None,...,None,None,None,None,None,None,None,Inf,Act,False
3,1,31,36,жизнь,1_4,1_3,obj,NOUN,жизнь,Inan,...,Acc,None,Fem,None,Sing,None,None,None,None,False
4,1,37,40,мою,1_5,1_17,det,DET,мой,None,...,Acc,None,Fem,None,Sing,None,None,None,None,True
5,1,42,49,времени,1_7,1_10,obj,NOUN,время,Inan,...,Gen,None,Neut,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47480,2360,283,291,великого,1_49,1_50,amod,ADJ,великий,None,...,Gen,Pos,Neut,None,Sing,None,None,None,None,False
47481,2360,292,296,дела,1_50,1_11,obl,NOUN,дело,Inan,...,Gen,None,Neut,None,Sing,None,None,None,None,False
47482,2360,300,310,всемирного,1_52,1_53,amod,ADJ,всемирный,None,...,Gen,Pos,Masc,None,Sing,None,None,None,None,False


In [15]:
lemmaDf = bibleTokenDf[['lemma']].loc[bibleTokenDf.stopword == False].join(motherTokenDf[['lemma']].loc[motherTokenDf.stopword == False], on='id', lsuffix='_bible', rsuffix='_mother')
lemmaDf

,lemma_bible,lemma_mother
id,,
2,начало,день
3,сотворить,NaN
4,бог,рабочий
5,небо,слободка
7,земля,дымный
...,...,...
675060,благодать,NaN
675061,господь,NaN
675063,иисус,NaN


## Shared Lemmas

In [16]:
bible_lemmas = bibleTokenDf.loc[bibleTokenDf.stopword == False].lemma
mother_lemmas = motherTokenDf.loc[motherTokenDf.stopword == False].lemma
confession_lemmas = confessionTokenDf.loc[confessionTokenDf.stopword == False].lemma

In [17]:
bible_vc = bible_lemmas.to_frame().value_counts().to_frame()
mother_vc = mother_lemmas.to_frame().value_counts().to_frame()

In [18]:
bible_vc.join(mother_vc, lsuffix='_bible', rsuffix='_mother')

,count_bible,count_mother
lemma,,
господь,7080,24.0
сказать,5844,697.0
сын,5415,187.0
бог,4376,55.0
говорить,3732,569.0
...,...,...
поворотили,1,NaN
договариваться,1,NaN
повредились,1,NaN


In [16]:
def ListSplit(lst, numGroups, sort=True): 
    """Takes a list and a number and splits the 
    list into evenly divided n groups (as much as possible)"""
    # choose to sort list by ascending
    if sort: 
        lst.sort()
    # get length of list given to sort
    listLen = len(lst)
    # groupLen is the maximum number of items per group to allow for the most groups <= numGroups in a list (numGroups-1 OR numGroups)
    groupLen = (listLen//numGroups) + (listLen % numGroups > 0)
    # yield generator object of nested listed with length of numGroups 
    for i in range(0,len(lst), groupLen): 
        yield lst[i:i+groupLen]

def token_window(lemma_array, window_size):
    token_window_array = np.array([])
    token_window_array = np.array([np.array(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def token_window_mp(lemma_array, window_size):
    #token_window_array = np.array([])
    token_window_array = enumerate([list(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def get_sim_count(child_window_array, parent_window_array, order_matters=False):
    work_sim_count = 0
    work_sim = []
    for child_window in child_window_array:
        child_sim = {}
        #print(f"child window: {child_window}")
        child_window_sim_total_count = 0
        for parent_window in parent_window_array:
            #print(f"parent window: {parent_window}")
            child_window_sim_sub_count = 0
            for i in range(len(child_window)):
                if order_matters:
                    if child_window[1][i] == parent_window[1][i]:
                        child_window_sim_total_count+=1
                        child_window_sim_sub_count+=1
                else:
                    if child_window[1][i] in parent_window[1]:
                        #print(f"{child_window[i]} is in parent!")
                        child_window_sim_total_count+=1
                        child_window_sim_sub_count+=1
                    #else:
                        #print(f"{child_window[i]} is not in parent!")
            child_sim.update({parent_window[0]: child_window_sim_sub_count})
        #print(f"{child_window}: {child_window_sim_total_count}")
        work_sim.append({child_window_sim_total_count: child_sim})
    return work_sim

def dna_test_windows(child_lemmas, parent_lemmas, window_size=10):
    sim_counter = 0
    #sim_list = np.array([])
    parent_window_array = token_window(parent_lemmas, window_size)
    child_window_array = token_window(child_lemmas, window_size)
    sim_list = get_sim_count(child_window_array, parent_window_array)
    #print(sim_list)
    return sim_list

def dna_test_windows_mp(child_lemmas, parent_lemmas, window_size=10):
    #child_lemmas, parent_lemmas = cnp_lemmas[0], cnp_lemmas[1]
    #print(f"Child lemmas: {child_lemmas}")
    #print(f"Parent lemmas: {parent_lemmas[:10]}")
    sim_counter = 0
    sim_list = np.array([])
    parent_window_array = token_window_mp(parent_lemmas, window_size)
    child_window_array = token_window_mp(child_lemmas, window_size)
    sim_list = get_sim_count(child_window_array, parent_window_array)
    return sim_list

def window_count_sim_mp(child_window_array, parent_window_array, order_matters=False):
    sim_count_tuples = {}
    for (c_sub_array_id, c_sub_array_list) in child_window_array:
        c_sub_sim_count = 0
        for (p_sub_array_id, p_sub_array_list) in parent_window_array:
            c_p_sim_count = 0
            for i in range(len(c_sub_array_list)):
                if order_matters:
                    if c_sub_array_list[i] == p_sub_array_list[i]:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
                else:
                    if c_sub_array_list[i] in p_sub_array_list:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
            sim_count_tuples[(c_sub_array_id, p_sub_array_id)] = c_p_sim_count
    return sim_count_tuples

In [30]:
def list_split(lst, numGroups, sort=True): 
    """Takes a list and a number and splits the 
    list into evenly divided n groups (as much as possible)"""
    # choose to sort list by ascending
    if sort: 
        lst.sort()
    # get length of list given to sort
    listLen = len(lst)
    # groupLen is the maximum number of items per group to allow for the most groups <= numGroups in a list (numGroups-1 OR numGroups)
    groupLen = (listLen//numGroups) + (listLen % numGroups > 0)
    # yield generator object of nested listed with length of numGroups 
    for i in range(0,len(lst), groupLen): 
        yield lst[i:i+groupLen]

def token_window_mp(lemma_array, window_size):
    #token_window_array = np.array([])
    token_window_array = enumerate([list(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def window_count_sim_mp(child_window_array, parent_window_array, order_matters=False):
    sim_count_tuples = {}
    for (c_sub_array_id, c_sub_array_list) in child_window_array:
        c_sub_sim_count = 0
        for (p_sub_array_id, p_sub_array_list) in parent_window_array:
            c_p_sim_count = 0
            for i in range(len(c_sub_array_list)):
                if order_matters:
                    if c_sub_array_list[i] == p_sub_array_list[i]:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
                else:
                    if c_sub_array_list[i] in p_sub_array_list:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
            if order_matters: 
                if c_p_sim_count > 1: 
                    sim_count_tuples[(c_sub_array_id, p_sub_array_id)] = c_p_sim_count
            elif c_p_sim_count > 2: 
                sim_count_tuples[(c_sub_array_id, p_sub_array_id)] = c_p_sim_count
    return sim_count_tuples

def dna_test_windows_mp(child_lemmas, parent_lemmas, window_size=10):
    sim_list = {}
    parent_window_array = token_window_mp(parent_lemmas, window_size)
    child_window_array = token_window_mp(child_lemmas, window_size)
    sim_list = window_count_sim_mp(child_window_array, parent_window_array)
    return sim_list

In [18]:
child_window_array = list(token_window_mp(mother_lemmas, 10))
parent_window_array = list(token_window_mp(bible_lemmas, 10))

In [19]:
len(child_window_array)

54729

In [20]:
len(parent_window_array)

373398

In [21]:
%%time
if __name__ == "__main__":
    pool = Pool(processes=num_cores)
    #workerAssignments = ((x, bible_lemmas) for x in ListSplit(mother_lemmas[:198], num_cores, False))
    starmap_list = list(zip(list_split(child_window_array, num_cores), list_split(parent_window_array, num_cores)))
    print(f"Splitting into {num_cores} groups")
    start_time = time.time()
    sim_tuples = pool.starmap(window_count_sim_mp, starmap_list, chunksize=2)
    print(f"Finished in {round(time.time()-start_time, 3)} seconds.")
    pool.close()
    pool.join()
    #print(sim_tuples)

Splitting into 38 groups
Finished in 22.431 seconds.
CPU times: user 1.19 s, sys: 1.05 s, total: 2.23 s
Wall time: 23.3 s


In [22]:
for i in range(len(sim_tuples)):
    print(f"len({i}): {len(sim_tuples[i])}")

len(0): 151185
len(1): 236023
len(2): 108737
len(3): 108898
len(4): 131523
len(5): 201457
len(6): 126186
len(7): 213333
len(8): 190028
len(9): 217626
len(10): 191311
len(11): 159041
len(12): 99477
len(13): 150578
len(14): 150077
len(15): 129582
len(16): 131736
len(17): 111190
len(18): 96336
len(19): 134694
len(20): 95853
len(21): 124034
len(22): 168739
len(23): 137390
len(24): 123513
len(25): 152575
len(26): 102194
len(27): 143978
len(28): 87372
len(29): 89773
len(30): 133872
len(31): 191409
len(32): 152113
len(33): 187804
len(34): 157302
len(35): 111830
len(36): 111003
len(37): 112023


In [25]:
motherTokenDf.loc[motherTokenDf.lemma == "эммаус"]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
2520,78,40,46,Эммаус,1_7,1_5,obl,PROPN,эммаус,Inan,...,Loc,None,Masc,None,Sing,None,None,None,None,False
25922,1351,857,863,Эммаус,5_68,5_66,nmod,PROPN,эммаус,Inan,...,Loc,None,Masc,None,Sing,None,None,None,None,False


In [24]:
total_consec_sim_tuples = {}
for t in sim_tuples:
    total_consec_sim_tuples.update(t)

In [25]:
len(total_consec_sim_tuples)

5421795

In [33]:
pd.DataFrame(child_window_array[8250][1])

,0
0,по-кочегарски
1,бог
2,подобный
3,огонь
4,так
5,жить
6,сердце
7,сказать
8,бог
9,слово


In [42]:
pd.DataFrame.from_dict({x:y for x,y in parent_window_array[:5]})

,0,1,2,3,4
0,начало,сотворить,бог,небо,земля
1,сотворить,бог,небо,земля,земля
2,бог,небо,земля,земля,безвидный
3,небо,земля,земля,безвидный,пустой
4,земля,земля,безвидный,пустой,тьма
5,земля,безвидный,пустой,тьма,бездна
6,безвидный,пустой,тьма,бездна,дух
7,пустой,тьма,бездна,дух,божий
8,тьма,бездна,дух,божий,носиться
9,бездна,дух,божий,носиться,вода


In [53]:
parent_window_array[57300]

(57300,
 ['имя',
  'взыскать',
  'пророк',
  'дерзнуть',
  'говорить',
  'имя',
  'повелеть',
  'говорить',
  'говорить',
  'имя'])

In [43]:
sorted_sim_tuples = sorted(total_sim_tuples.items(), key=lambda x:x[1], reverse=True)

In [26]:
sorted_consec_sim_tuples = sorted(total_consec_sim_tuples.items(), key=lambda x:x[1], reverse=True)

In [27]:
sorted_consec_sim_tuples

[((2284, 17791), 3),
 ((2285, 17792), 3),
 ((2286, 17793), 3),
 ((2348, 13927), 3),
 ((2355, 13399), 3),
 ((2356, 13400), 3),
 ((2357, 13401), 3),
 ((2515, 12268), 3),
 ((2516, 12269), 3),
 ((2517, 12270), 3),
 ((2518, 12271), 3),
 ((4055, 21854), 3),
 ((4056, 21855), 3),
 ((4908, 37762), 3),
 ((5119, 36624), 3),
 ((5120, 36625), 3),
 ((5121, 36626), 3),
 ((5122, 36627), 3),
 ((5123, 36628), 3),
 ((5124, 36629), 3),
 ((5125, 36630), 3),
 ((5126, 36631), 3),
 ((5784, 40092), 3),
 ((5785, 40093), 3),
 ((8163, 53086), 3),
 ((8164, 53087), 3),
 ((8165, 53088), 3),
 ((8166, 53089), 3),
 ((8167, 53090), 3),
 ((8168, 53091), 3),
 ((8169, 53092), 3),
 ((8170, 53093), 3),
 ((8255, 57313), 3),
 ((8256, 57314), 3),
 ((8257, 57315), 3),
 ((8265, 52937), 3),
 ((8265, 55011), 3),
 ((8266, 52938), 3),
 ((8266, 55012), 3),
 ((8267, 55013), 3),
 ((8268, 55014), 3),
 ((8295, 54650), 3),
 ((8296, 54651), 3),
 ((8297, 54652), 3),
 ((8298, 54653), 3),
 ((8315, 52656), 3),
 ((10696, 76630), 3),
 ((10697, 76

In [31]:
compareDf = pd.DataFrame()
for x,y in sorted_sim_tuples[10:20]:
    #print(child_window_array[x[0]][0])
    #print(child_window_array[x[0]][1])
    compareDf[child_window_array[x[0]][0]] = child_window_array[x[0]][1]
    compareDf[parent_window_array[x[1]][0]] = parent_window_array[x[1]][1]
    #compareDf[x[1]] = parent_window_array[x][1]

compareDf

NameError: name 'sorted_sim_tuples' is not defined

In [45]:
sorted_sim_tuples

[((18065, 123203), 7),
 ((18065, 123204), 7),
 ((18065, 123205), 7),
 ((18065, 123206), 7),
 ((18065, 124132), 7),
 ((18065, 124133), 7),
 ((18065, 124134), 7),
 ((18065, 124135), 7),
 ((18065, 124136), 7),
 ((18065, 124137), 7),
 ((18065, 124138), 7),
 ((19783, 135289), 7),
 ((19783, 135290), 7),
 ((6157, 43527), 6),
 ((6157, 43528), 6),
 ((6157, 43529), 6),
 ((6157, 43530), 6),
 ((6157, 43531), 6),
 ((6157, 43532), 6),
 ((6157, 43533), 6),
 ((6157, 43534), 6),
 ((6157, 43535), 6),
 ((6157, 46262), 6),
 ((6157, 46263), 6),
 ((6157, 46264), 6),
 ((6157, 46265), 6),
 ((6157, 46266), 6),
 ((6157, 46267), 6),
 ((6157, 46268), 6),
 ((6157, 46308), 6),
 ((8251, 53357), 6),
 ((8251, 57309), 6),
 ((8251, 57310), 6),
 ((8255, 57309), 6),
 ((8255, 57310), 6),
 ((8256, 57309), 6),
 ((8256, 57310), 6),
 ((18063, 123203), 6),
 ((18063, 123204), 6),
 ((18063, 123205), 6),
 ((18063, 123206), 6),
 ((18063, 124132), 6),
 ((18063, 124133), 6),
 ((18063, 124134), 6),
 ((18063, 124135), 6),
 ((18063, 124

In [44]:
with open('./proc/sim_tuples.pkl', 'wb') as file:
    pickle.dump(sorted_sim_tuples, file)

In [32]:
with open('./proc/consec_sim_tuples.pkl', 'wb') as file:
    pickle.dump(sorted_consec_sim_tuples, file)

In [21]:
with open('./proc/sim_tuples.pkl', 'rb') as file:
    # The protocol version used is detected automatically, so we do not have to specify it.
    sim_tuples = pickle.load(file)

In [23]:
sim_tuples

{(0, 14): 1,
 (0, 15): 1,
 (0, 16): 1,
 (0, 17): 1,
 (0, 18): 1,
 (0, 19): 1,
 (0, 20): 1,
 (0, 21): 1,
 (0, 22): 1,
 (0, 23): 1,
 (0, 24): 1,
 (0, 25): 1,
 (0, 26): 1,
 (0, 27): 1,
 (0, 28): 1,
 (0, 29): 1,
 (0, 30): 1,
 (0, 31): 1,
 (0, 32): 1,
 (0, 33): 1,
 (0, 34): 1,
 (0, 35): 1,
 (0, 36): 1,
 (0, 37): 1,
 (0, 38): 1,
 (0, 50): 1,
 (0, 51): 1,
 (0, 52): 1,
 (0, 53): 1,
 (0, 54): 1,
 (0, 55): 1,
 (0, 56): 1,
 (0, 57): 1,
 (0, 58): 1,
 (0, 59): 1,
 (0, 60): 1,
 (0, 61): 1,
 (0, 62): 1,
 (0, 63): 1,
 (0, 64): 1,
 (0, 65): 1,
 (0, 66): 1,
 (0, 67): 1,
 (0, 68): 1,
 (0, 69): 1,
 (0, 122): 1,
 (0, 123): 1,
 (0, 124): 1,
 (0, 125): 1,
 (0, 126): 1,
 (0, 127): 1,
 (0, 128): 1,
 (0, 129): 1,
 (0, 130): 1,
 (0, 131): 1,
 (0, 132): 1,
 (0, 133): 1,
 (0, 134): 1,
 (0, 135): 1,
 (0, 136): 1,
 (0, 137): 1,
 (0, 138): 1,
 (0, 139): 1,
 (0, 140): 1,
 (0, 141): 1,
 (0, 142): 1,
 (0, 143): 1,
 (0, 144): 1,
 (0, 145): 1,
 (0, 146): 1,
 (0, 147): 1,
 (0, 148): 1,
 (0, 149): 1,
 (0, 150): 1,
 (0, 151)

In [ ]:
pd.DataFrame(sim_tuples)

How do Gor'kii's *Mother* and the Synodal Bible overlap? 
windows of 3-5 words with 1-2+ of overlap